In [2]:
import numpy as nump
import pandas as pd
import numpy as np
import seaborn as sea
import datetime as dt

In [3]:
class RNN:
    #(Input, Saida Experada(em um passo), Numero de recorrencias,vetor de Saidas Experadas, escala de aprendizado)
    def __init__ (self,input,output, recorrencia, saidas_experadas, escala):
        self.x = np.zeros(input)
        self.input = input
        self.y = nump.zeros(output)
        self.output = output
        #matriz de peso para a interpretação das celulas "hidden state"
        self.w = np.random.random((output,output))
        #matriz para ordenar a escala de aprendizado
        self.g = nump.zeros_like(self.w)
        #numero de recorrencias
        self.recorrencia = recorrencia
        #escala de aprendizado
        self.lr = escala
        #arquivando inputs
        self.ia = nump.zeros((recorrencia + 1, input))
        #arquivando estados de celulas
        self.ca = nump.zeros((recorrencia + 1, output))
        #arquivando os outputs
        self.oa = nump.zeros((recorrencia + 1, output))
        #arquivando para as hidden states
        self.ha = nump.zeros((recorrencia + 1, output))
        #portão de esquecimento
        self.af = nump.zeros((recorrencia + 1, output))
        #portao de input
        self.ai = nump.zeros((recorrencia + 1, output))
        #portao Celula
        self.ac = nump.zeros((recorrencia + 1, output))
        #portao de output
        self.ao = nump.zeros((recorrencia + 1, output))
        #vetor output
        self.saidas_experadas = nump.vstack((np.zeros(saida_experadas[0]),saidas_experadas.T))
        #Criação da celula LSTM
        self.LSTM = (input,output,recorrencia,escala)

In [4]:
    #funçao de ativação
    def sigmoid(self, x):
        return 1/(1+nump.exp(-x))
        
    #derivada da Sigmoid
    def dsigmoid(self,x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
        
    #Matriz de operações através do Input, para computar o Output
    def forwardProp(self):
        for i in range(1+self.recorrencia):
            self.LSTM.x = nump.hstack((self.ha[i-1],self.x))
            cs, hs, f, c, o = self.LSTM.forwardProp()
            self.ca[i] = cs #estado da celula
            self.ha[i] = hs #hidden state
            self.af[i] = f  #forget state
            self.ai[i] = inp
            self.ac[i] = c
            self.ao[i] = o #output
            self.oa[i] = self.sigmoid(nump.dot(self.w,hs))
            self.x = self.saidas_experadas[I-1]
        return self.oa
    #Backpropagation dos pesos das matrizes
    def backProp(self):
        totalError = 0
        #Inicializando as matrizes de atualizações gradientes
        #Estado das celulas
        dfcs = nump.zeros(self.output)
        #Hidden State
        dfhs = nump.zeros(self.output)
        #Matriz Peso
        tu = nump.zeros((self.output,self.output))
        #Matriz de esquecimento
        tfu = nump.zeros((self.output,self.input+self.output))
        #Input Gate
        tiu = nump.zeros((self.output,self.input+self.output))
        #Celula
        tcu = nump.zeros((self.output,self.input+self.output))
        #Output Gate
        tou = nump.zeros((self.output,self.input+self.output))
        #Loop através das recorrencias
        for i in range(self.recorrencia, -1, -1):
            #erro = saida calculada - saida esperada
            erro = self.oa[i] - self.saidas_experadas[i]
            #calcular e atualizar matriz peso
            tu += np.dot(nump.atleast_2d(erro * self.dsigmoid(self.oa[i])),nump.atleast_2d(self.ha[i]).T)
            #Propagaar o erro até a saida da função
            erro = nump.dot(erro,self.w)
            self.LSTM.x = nump.hstack((self.ha[i-1], self.ia[i]))
            self.LSTM.cs = self.ca[i]
            fu,iu,cu,ou,dfcs,dfhs = self.LSTM.backProp(erro,self.ca[i-1],self.af[i],self.ai[i],self.ac[i],self.ao[i],dfcs,dfhs)
            totalError += nump.sum(erro)
            tfu += fu
            tiu += iu
            tcu += cu
            tou += ou
        self.LSTM.update(tfu/self.recorrencia,tiu/self.recorrencia,tcu/self.recorrencia,tou/self.recorrencia)
        self.update(tu/self.recorrencia)
        return totalError
    
    def update(self,u):
        self.G = 0.95 * self.G + 0.1 * u**2
        self.w -= self.escala/nump.sqrt(self.G + 1e-8) * u
        return
    
    def sample(self):
        for i in range(1, self.recorrencia+1):
            self.LSTM.x = nump.hstack((self.ha[i-1], self.x))
            cs,hs,f,inp,c,o = self.LSTM.forwardProp()
            maxI = nump.argmax(self.x)
            seld.x = nump.zeros_like(self.x)
            self.x[maxI] = 1
            self.ia[i] = self.x
            self.ca[i] = cs
            self.ha[i] = hs
            self.af[i] = f
            self.ac[i] = c
            self.ao[i] = o
            self.oa[i] = self.sigmoid(nump.dot(self.w,hs))
            MaxI = nump.argmax(self.oa[i])
            newX = nump.zeros_like(self.x)
            newX[maxI] = 1
            self.x = newX
        return self.oa

In [5]:
class LSTM:
    def __init__ (self, input, output, recorrencia, escala):
        self.x = nump.zeros(input+output)
        self.input = input + output
        self.y = nump.zeros(output)
        self.output = output
        self.cs = nump.zeros(output)
        self.recorrencia = recorrencia
        self.escala = escala
        #Portoes de Esquecimento, Input, Output e Celula
        self.f = nump.random.random((output,input+output))
        self.i = nump.random.random((output,input+output))
        self.c = nump.random.random((output,input+output))
        self.o = nump.random.random((output,input+output))
        self.Gf = nump.zeros_like(self.f)
        self.Gi = nump.zeros_like(self.i)
        self.Gc = nump.zeros_like(self.c)
        self.Go = nump.zeros_like(self.o)
    def sigmoid(self, x):
        return 1/(1 + numo.exp(-x))
    
    def dsigmoid(self,x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    def tangent(self,x):
        return nump.tanh(x)
    
    def dtangent(self,x):
        return 1 - nump.tanh(x)**2
    
    def forwardProp(self):
        f = self.sigmoid(nump.dot(self.f,self.x))
        self.cs *= f
        i = self.sigmoid(nump.dot(self.i,self.x))
        c = self.sigmoid(nump.dot(self.c,self.x))
        self.cs += i *c
        o = self.sigmoid(nump.dot(self.o,self.x))
        self.y = o * self.tangent(self.cs)
        return self.cs, self.y, f, i, c, o
    
    def backProp(self, e, pcs,f,i,c,o,dfcs,dfhs):
        e = nump.clip(e + dfhs, -6, 6)
        do = self.tangent(self.cs) * e
        ou = nump.dot(nump.atleast_2d(do * self.dtangent(o)).T, nump.atleast_2d(self.x))
        dcs = nump.clip(e * o * self.dtangent(self.cs) +dfcs, -6, 6)
        dc = dcs * i
        cu = nump.dot(nump.atleast_2d(dc * self.dtangent(c)).T,nump.atleast_2d(self.x))
        di = dcs * c
        iu = nump.dot(nump.atleast_2d(di * self.dsigmoid(i)).T,nump.atleast_2d(self.x))
        df = dcs * pcs
        fu = nump.dot(nump.atleast_2d(df * self.dsigmoid(f)).T,nump.atleast_2d(self.x))
        dpcs = dcs * f
        dphs = nump.dot(dc,self.c)[:self.output] + nump.dot(do,self.o)[:self.output] + nump.dot(di,self.i)[:self.output] + nump.dot(df,self.f)[:self.output]
        return fu,iu,cu,ou,dpcs,dphs
    
    def update(self,fu,iu,cu,ou):
        self.Gf = 0.9 * self.Gf + 0.1 * fu**2
        self.Gi = 0.9 * self.Gi + 0.1 * iu**2
        self.Gc = 0.9 * self.Gc + 0.1 * cu**2
        self.Go = 0.9 * self.Go + 0.1 * ou**2
        
        self.f -= self.escala/nump.sqrt(self.Gf + 1e-8) * fu
        self.i -= self.escala/nump.sqrt(self.Gi + 1e-8) * iu
        self.c -= self.escala/nump.sqrt(self.Gc + 1e-8) * cu
        self.o -= self.escala/nump.sqrt(self.Go + 1e-8) * ou
        return

In [23]:
#Codigo referente aos trabalhos com o Dataset 
def LoadDataSet_newCases():
    coronga = pd.read_csv('base.csv')
    coronga = coronga.fillna({'continent':"No Continent",'total_cases':0, 'new_cases':0,
           'total_deaths':0, 'new_deaths':0, 'population':0, 'population_density':0,
           'median_age':0,'new_tests':0,'median_age':50,'life_expectancy':0})
    
    #Criação de Colunas novas em relação à datas
    coronga['date'] = pd.to_datetime(coronga['date'])
    coronga['day'] = coronga['date'].dt.day
    coronga['month'] = coronga['date'].dt.month
    coronga['year'] = coronga['date'].dt.year
    #Trimestre,Semestre,bimestre
    coronga['quarter'] = coronga['date'].dt.quarter
    coronga['half'] = np.where(coronga['month']>6, 2, 1)
    #Condições
    cond = [
        (coronga['month'] <= 2),
        (coronga['month'] > 2) & (coronga['month'] <= 4),
        (coronga['month'] > 4) & (coronga['month'] <= 6),
        (coronga['month'] > 6) & (coronga['month'] <= 8),
        (coronga['month'] > 8) & (coronga['month'] <= 10),
        (coronga['month'] > 10)
        ]
    
    # Lista de valores para cada condição
    vl = [1,2,3,4,5,6]
    
    # Nova coluna baseada nos valores anteriores
    coronga['bimonth'] = np.select(cond, vl)
    
    #Criando Codigos para as Categoricas
    coronga['continent'] = coronga['continent'].astype('category')
    # Criando os codigos de Continente
    coronga['cod_continent'] = coronga['continent'].cat.codes
    coronga['location'] = coronga['location'].astype('category')
    # Criando os codigos de Local
    coronga['cod_location'] = coronga['location'].cat.codes
    
    #exportar Tabelas relacionadas.
    local = coronga['location']
    continentes = coronga['continent']
    cd_local = coronga['cod_location']
    cd_continentes = coronga['cod_continent']
    data_local = {'cod':cd_local.unique(),
                  'location':local.unique()}
    data_continent = {'cod':cd_continentes.unique(),
                  'continent':continentes.unique()}
    df_local = pd.DataFrame(data_local)
    df_continent = pd.DataFrame(data_continent)
    df_local.to_csv('local.csv')
    df_continent.to_csv('continentes.csv')
    
    #Criação de colunas previous Cases and Deaths
    coronga['prev_cases'] = coronga.groupby('cod_location')['new_cases'].shift(-1)
    coronga['prev_deaths'] = coronga.groupby('cod_location')['new_deaths'].shift(-1)
    
    #droppagem de colunas não utilizaveis
    dropped = ['iso_code','date','total_cases_per_million','new_cases_per_million','total_deaths_per_million',
           'new_deaths_per_million', 'total_tests',
           'total_tests_per_thousand', 'new_tests_per_thousand',
           'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'tests_units',
           'stringency_index', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
           'cvd_death_rate', 'diabetes_prevalence', 'female_smokers',
           'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand','continent','location']
    
    coronga.drop(dropped,inplace=True,axis=1)
    coronga = coronga.dropna()
    #Saida em .csv
    coronga.to_csv('in.csv')
    outputSize = coronga.size
    data = set(coronga)
    uniqueWords, dataSize = len(data), len(data) 
    returnData = np.zeros((uniqueWords, dataSize))
    for i in range(0, dataSize):
        returnData[i][i] = 1
    returnData = np.append(returnData, np.atleast_2d(data), axis=0)
    output = np.zeros((uniqueWords, outputSize))
    for i in range(0, outputSize):
        index = np.where(np.asarray(data) == text[i])
        output[:,i] = returnData[0:-1,index[0]].astype(float).ravel()  
    return returnData, uniqueWords, output, outputSize, data
    ##########################################################################

In [25]:
iterations = coronga.size
learningRate = 0.001
returnData, numCategories, expectedOutput,outputSize, data = LoadDataSet_newCases()

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 19 and the array at index 1 has size 1